# @robotodio - tweet extractor

---

## Libraries

In [1]:
# Python libraries
# ------------------------------------------------------------------------------
# Reading files with different formats
import json

# Data wrangling
import re
import pandas as pd
import numpy as np

# Twitter API
import tweepy

# NLP
from textblob import TextBlob

# Hate speech detection
from detoxify import Detoxify

## Download data from twitter

The tweepy API class provides access to the entire Twitter RESTful API methods. Each method can accept various parameters and return responses.

### API set up

In [2]:
# API Twitter credentials
# ------------------------------------------------------------------------------
# Open .json file containing credentials/tokens as a dictionary
with open("twitter_api_keys.json") as file:
    api_credentials = json.load(file)
    
# Assign each value of the dictionary to a new variable
consumer_key = api_credentials['consumer_key']
consumer_secret = api_credentials['consumer_secret']
access_token = api_credentials['access_token']
access_token_secret = api_credentials['access_token_secret']

In [3]:
# API set up
# ------------------------------------------------------------------------------
# Create a handler instance with key and secret consumer, and pass the tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
    
# Construct the API instance
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Check credentials
if(api.verify_credentials):
    print("Logged In Successfully")
else:
    print("Error -- Could not log in with your credentials")

Logged In Successfully


### Extract tweets and associated metadata

Once the `tweepy.API` class has been instantiated we can call its methods to scrap desired information in Twitter.

Pagination is used a lot in Twitter API development. Iterating through timelines, user lists, direct messages, etc. In order to perform pagination, we must supply a page/cursor parameter with each of our requests. The problem here is this requires a lot of boiler plate code just to manage the pagination loop. To help make pagination easier and require less code, Tweepy has the Cursor object.

In first place, we are going to define a function that returns an iterator of tweets. We will call this fuction each time we want to download tweets from an user account.

In [4]:
# Tweets list iterator
# ------------------------------------------------------------------------------
def tweets_iterator(target, n_items):
    '''
    Returns an iterator of tweets.

        Parameters:
            target (str): The user name of the Twitter account.
            n_items (int): Number of tweets downloaded.

        Returns:
            tweets (ItemIterator): an iterator of tweets
    '''
    # Instantiate the iterator
    tweets = tweepy.Cursor(
        api.user_timeline,
        screen_name=target,
        include_rts=False,
        exclude_replies=False,
        tweet_mode='extended').items(n_items)
    
    # Returns iterator
    return tweets

In the next cell we can see an example of all the metadata that contains one tweet.

In [5]:
# Introduce the target Twitter account and number of items to download
target = 'DFD_74'
n_items = 20

# Tweets list (iterator)
tweets = tweets_iterator(target, n_items)

for k, tweet in enumerate(tweets):
    print(json.dumps(tweet._json, indent=4))
    if k == 1:
        break

{
    "created_at": "Mon Feb 22 15:35:58 +0000 2021",
    "id": 1363875185678499844,
    "id_str": "1363875185678499844",
    "full_text": "Otro ejemplo de la cobard\u00eda del PP \nNo valen para nada https://t.co/7DLqlYtyfz",
    "truncated": false,
    "display_text_range": [
        0,
        54
    ],
    "entities": {
        "hashtags": [],
        "symbols": [],
        "user_mentions": [],
        "urls": [
            {
                "url": "https://t.co/7DLqlYtyfz",
                "expanded_url": "https://twitter.com/ipeutrera/status/1363832249163132933",
                "display_url": "twitter.com/ipeutrera/stat\u2026",
                "indices": [
                    55,
                    78
                ]
            }
        ]
    },
    "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_us

As we can see in the previous output cell, each tweet contains a lot of metadata. All this information comes in .json format, and we can access to it with Tweepy methods, or with dictionary methods.

In the next cell we are going to extract some metadata from each tweet, and export to a Pandas DataFrame format.

In [107]:
# Tweet extractor
# ------------------------------------------------------------------------------
# Tweets list (iterator)
tweets = tweets_iterator(target, n_items)

# Read through the iterator, and export the info to a Pandas DataFrame
all_columns = [np.array([
    tweet.full_text,
    tweet.user.screen_name,
    tweet.id,
    tweet.source,
    tweet.created_at,
    len(tweet.full_text),
    tweet.favorite_count,
    tweet.retweet_count,
    str(tweet.entities['hashtags'])
]) for tweet in tweets]

# Export the list of tweets to a dataframe
df = pd.DataFrame(
    data=all_columns,
    columns=['tweet', 'id', 'account', 'source', 'date', 'length', 'likes',
             'RTs', 'hashtags']
)

df.head()

,tweet,id,account,source,date,length,likes,RTs,hashtags
0,Otro ejemplo de la cobardía del PP \nNo valen ...,DFD_74,1363875185678499844,Twitter for Android,2021-02-22 15:35:58,78,0,0,[]
1,Una famosa anuncia que tiene cáncer \n¿Se lo d...,DFD_74,1363874709264334852,Twitter for Android,2021-02-22 15:34:04,129,1,1,[]
2,@LeaMicheleM Qué analfabeta 🤦‍♂️,DFD_74,1363873760105926656,Twitter for Android,2021-02-22 15:30:18,32,9,0,[]
3,"Si el 8M se autoriza algo, se acabaron las res...",DFD_74,1363873344311934978,Twitter for Android,2021-02-22 15:28:39,81,20,6,[]
4,@Rafa_Hernando Reunificación en torno a vox cu...,DFD_74,1363872371933855747,Twitter for Android,2021-02-22 15:24:47,102,5,0,[]


### Clean and translation

Once we have a dataset with all the tweets of an user, we need to prepare and clean them to serve as input to the model. For each tweet we are going to eliminate special string characters, and we will translate them into English in case they are not.

In [108]:
# Characters to remove
spec_chars = ['\n', '\t', '\r']

In [109]:
# Replace defined characters with a whitespace
for char in spec_chars:
    df['tweet'] = df['tweet'].str.strip().replace(char, ' ')

Because we replaced the special characters with a whitespace, we might end up with double whitespaces in some values. Let’s remove them by splitting each tweet using whitespaces and re-joining the words again using join.

In [110]:
# Split and re-join each tweet
df['tweet'] = df['tweet'].str.split().str.join(" ")

In [111]:
df.head()

,tweet,id,account,source,date,length,likes,RTs,hashtags
0,Otro ejemplo de la cobardía del PP No valen pa...,DFD_74,1363875185678499844,Twitter for Android,2021-02-22 15:35:58,78,0,0,[]
1,Una famosa anuncia que tiene cáncer ¿Se lo dec...,DFD_74,1363874709264334852,Twitter for Android,2021-02-22 15:34:04,129,1,1,[]
2,@LeaMicheleM Qué analfabeta 🤦‍♂️,DFD_74,1363873760105926656,Twitter for Android,2021-02-22 15:30:18,32,9,0,[]
3,"Si el 8M se autoriza algo, se acabaron las res...",DFD_74,1363873344311934978,Twitter for Android,2021-02-22 15:28:39,81,20,6,[]
4,@Rafa_Hernando Reunificación en torno a vox cu...,DFD_74,1363872371933855747,Twitter for Android,2021-02-22 15:24:47,102,5,0,[]


Finally we translate them into English. For carrying out this task, we are going to use TextBlob library, which is a module for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

In [112]:
# Translation example
text = "Esta frase está en español."
translated_text = TextBlob(texto).translate(to="en").string
translated_text

'This phrase is in Spanish.'

In [117]:
# Main language detection
# ------------------------------------------------------------------------------
# Detect the language of each tweet
df['org_language'] = df['tweet'].apply(
    lambda tweet: TextBlob(tweet).detect_language())

# Summary of the different languages
freq_languages = df[['id', 'org_language']].groupby(["org_language"])\
                                           .count()\
                                           .sort_values(["org_language"])\
                                           .reset_index()

# Most common language
main_language = freq_languages.values[0][0]
print(f"The most common language is {main_language}.")

The most common language is es.


In [114]:
# Translation
# ------------------------------------------------------------------------------
# Instance a np.array variable that contains all tweets
tweets = np.array(df['tweet'])

# We apply the TextBlob translation to each tweet of previous np.array
tweets_en = list(map(lambda tweet: TextBlob(tweet).translate(from_lang=main_language, to='en').string, tweets))

# Create another pandas column with the transalation values
df['tweet_en'] = tweets_en

In [116]:
df.head()

,tweet,id,account,source,date,length,likes,RTs,hashtags,org_language,tweet_en
0,Otro ejemplo de la cobardía del PP No valen pa...,DFD_74,1363875185678499844,Twitter for Android,2021-02-22 15:35:58,78,0,0,[],es,Another example of the PP's cowardice They are...
1,Una famosa anuncia que tiene cáncer ¿Se lo dec...,DFD_74,1363874709264334852,Twitter for Android,2021-02-22 15:34:04,129,1,1,[],es,A famous woman announces that she has cancer. ...
2,@LeaMicheleM Qué analfabeta 🤦‍♂️,DFD_74,1363873760105926656,Twitter for Android,2021-02-22 15:30:18,32,9,0,[],es,@LeaMicheleM How illiterate 🤦‍♂️
3,"Si el 8M se autoriza algo, se acabaron las res...",DFD_74,1363873344311934978,Twitter for Android,2021-02-22 15:28:39,81,20,6,[],es,"If the 8M something is authorized, the restric..."
4,@Rafa_Hernando Reunificación en torno a vox cu...,DFD_74,1363872371933855747,Twitter for Android,2021-02-22 15:24:47,102,5,0,[],es,@Rafa_Hernando Reunification around vox whenev...


## Hate speech level prediction

`Detoxify` library has three pre-trained models:

- original: toxic, severe_toxic, obscene, threat, insult, identity_hate.
- unbiased: toxicity, severe_toxicity, obscene, threat, insult, identity_attack, sexual_explicit.
- multilingual: toxicity

While *original* and *unbiased* models generate different scorings like: toxicity, obscene, threat, insult...etc, *multilingual* model only produces a toxicity scoring. On the other hand, *multilingual* model is the only one that can bring a toxicity score in a language different from english.

Like we want to analyze any kind of accounts independently from the origin, and in order to make a more robust and scalable solution, we are going to chose *multilingual* one.

In [57]:
# Hate speech level prediction
# ------------------------------------------------------------------------------
# Returns a dictionary with toxicity values of each tweet. The key of the
# dictionary is called 'toxicity'.
results = Detoxify('original').predict(list(df['tweet_en']))
results

{'toxicity': [0.34270909428596497,
  0.5066758990287781,
  0.5322281122207642,
  0.0006593515281565487,
  0.0012182380305603147,
  0.00709440978243947,
  0.026463046669960022,
  0.929384708404541,
  0.37596818804740906,
  0.715994656085968,
  0.0024967517238110304,
  0.3599565625190735,
  0.004273798782378435,
  0.0011286621447652578,
  0.9618690609931946,
  0.0010413483250886202,
  0.7519179582595825,
  0.1417776346206665,
  0.6257846355438232,
  0.020189963281154633],
 'severe_toxicity': [0.00036478773108683527,
  0.0019661018159240484,
  0.0005491891060955822,
  0.00011751129204640165,
  9.524037886876613e-05,
  0.0001000075280899182,
  0.00011255337449256331,
  0.01660938560962677,
  0.0003151522541884333,
  0.004687870386987925,
  8.849022560752928e-05,
  0.0003024043107870966,
  9.244104876415804e-05,
  9.703006799099967e-05,
  0.02406223490834236,
  0.00010246875899611041,
  0.0015902059385553002,
  0.0004606180009432137,
  0.003477138467133045,
  0.00011829930735984817],
 'obsc

In [62]:
# Export the new info to the previous DataFrame
df['toxicity'] = results['toxicity']
df['severe_toxicity'] = results['severe_toxicity']
df['obscene'] = results['obscene']
df['threat'] = results['threat']
df['insult'] = results['insult']
df['identity_hate'] = results['identity_hate']

df.head()

,tweet,id,account,source,date,length,likes,RTs,hashtags,org_language,tweet_en,toxicity,severe_toxicity,obscene,threat,insult,identity_hate
0,Otro ejemplo de la cobardía del PP No valen pa...,DFD_74,1363875185678499844,Twitter for Android,2021-02-22 15:35:58,78,0,0,[],es,Another example of the PP's cowardice They are...,0.342709,0.000365,0.004759,0.000578,0.035478,0.003897
1,Una famosa anuncia que tiene cáncer ¿Se lo dec...,DFD_74,1363874709264334852,Twitter for Android,2021-02-22 15:34:04,129,1,1,[],es,A famous woman announces that she has cancer. ...,0.506676,0.001966,0.004493,0.024383,0.029571,0.022523
2,@LeaMicheleM Qué analfabeta 🤦‍♂️,DFD_74,1363873760105926656,Twitter for Android,2021-02-22 15:30:18,32,5,0,[],es,@LeaMicheleM How illiterate 🤦‍♂️,0.532228,0.000549,0.004269,0.000617,0.037624,0.014153
3,"Si el 8M se autoriza algo, se acabaron las res...",DFD_74,1363873344311934978,Twitter for Android,2021-02-22 15:28:39,81,20,6,[],es,"If the 8M something is authorized, the restric...",0.000659,0.000118,0.000178,0.000119,0.000174,0.000139
4,@Rafa_Hernando Reunificación en torno a vox cu...,DFD_74,1363872371933855747,Twitter for Android,2021-02-22 15:24:47,102,4,0,[],es,@Rafa_Hernando Reunification around vox whenev...,0.001218,0.000095,0.000177,0.000110,0.000196,0.000143


## X. Bibliografía

- http://docs.tweepy.org/en/latest/
- https://github.com/unitaryai/detoxify

## X. Info a revisar

- Limpieza de datos con regex: http://rios.tecnm.mx/cdistribuido/recursos/MinDatScr/MineriaScribble.html
- Web objetivo: https://www.ninjalitics.com/


- Traduction --> TextBlob
- Sentiment Analysis --> TextBlob
- Universal Dependencies --> GSD

Preparar un traductor con TextBlob:

TextBlob también tiene análisis de opinión:

In [ ]:
# análisis de opinión
opinion1 = TextBlob("This new restaurant is great. I had so much fun!!")
print(opinion1.sentiment)

opinion2 = TextBlob("Google News to close in Spain.")
print(opinion2.sentiment)

# subjetividad 0:1
# polaridad -1:1


In [ ]:
from textblob.sentiments import NaiveBayesAnalyzer

sentences = [
    "I enjoy sports.",
    "Pizza and pasta are my favorite food.",
    "The hotel was horrible.",
    "The movie was awesome. Wonderful play!",
    "Worst actors I've seen in my whole life"
]

for sentence in sentences:
    t = TextBlob(sentence, analyzer=NaiveBayesAnalyzer())
    print(f"{sentence}\nsubj: {t.sentiment}\n")

Y corrección ortográfica:

In [ ]:
#  corrección ortográfica
b1 = TextBlob("I havv goood speling!")
print(b1.correct())

b2 = TextBlob("Miy naem iz Jonh!")
print(b2.correct())

b3 = TextBlob("Boyz dont cri")
print(b3.correct())

b4 = TextBlob("psicological posesion achifmen comitment")
print(b4.correct())